In [1]:
using Gen
using Serialization
using Logging
using BenchmarkTools

In [7]:
using GenArrow

┌ Debug: Rejecting stale cache file /Users/ian/.julia/compiled/v1.8/GenArrow/aNlSN_Vi0Tz.ji (mtime 1.6788300239800227e9) because file /Users/ian/Documents/probcomp/GenArrow.jl/src/GenArrow.jl (mtime 1.678830701971165e9) has changed
└ @ Base loading.jl:2150
┌ Debug: Rejecting cache file /Users/ian/.julia/compiled/v1.8/GenArrow/aNlSN_aNLwd.ji because module Parsers [69de0a69-1ddd-5017-9359-2bf0b02dc9f0] is already loaded and incompatible.
└ @ Base loading.jl:2100
┌ Debug: Rejecting cache file /Users/ian/.julia/compiled/v1.8/GenArrow/aNlSN_8wlmW.ji because module Parsers [69de0a69-1ddd-5017-9359-2bf0b02dc9f0] is already loaded and incompatible.
└ @ Base loading.jl:2100
┌ Debug: Rejecting cache file /Users/ian/.julia/compiled/v1.8/GenArrow/aNlSN_xGnfe.ji because module IrrationalConstants [92d709cd-6900-40b7-9082-c6be49f344b6] is already loaded and incompatible.
└ @ Base loading.jl:2100
┌ Debug: Precompiling GenArrow [38ed6c62-afb3-4a05-8af9-4104512be020]
└ @ Base loading.jl:1662


┌ Debug: Loading cache file /Users/ian/.julia/compiled/v1.8/GenArrow/aNlSN_Vi0Tz.ji for GenArrow [38ed6c62-afb3-4a05-8af9-4104512be020]
└ @ Base loading.jl:806


In [8]:
function write_to_file(fname, input)
    seekstart(input)
    data = read(input, String)
    open(fname, "w") do io
        write(io, data)
    end
end
function read_from_file(fname)
    io = IOBuffer()
    open(fname, "r") do file
        data = read(file, String)
        write(io, data)
    end
    seekstart(io)
    io
end

read_from_file (generic function with 1 method)

### Leaf Nodes

In [9]:
@gen function submodel(w)
    a ~ bernoulli(0.5)
    b ~ bernoulli(0.5)
end

@gen function model(n)
    z ~ bernoulli(0.5)
    # for k=1:n
    #     @trace(bernoulli(0.5), k)
    # end
    q ~ submodel("what the")
end

DynamicDSLFunction{Any}(Dict{Symbol, Any}(), Dict{Symbol, Any}(), Type[Any], false, Union{Nothing, Some{Any}}[nothing], var"##model#315", Bool[0], false)

In [10]:
debugIO = open("write.txt", "w+")
logger = ConsoleLogger(debugIO, Debug)
global_logger(logger)
tr_old, w_old = generate(model, (10,))
display(get_choices(tr_old))
display(get_score(tr_old))
io = IOBuffer()
GenArrow.serialize(io, tr_old)
close(debugIO)
write_to_file("data.trace", io)

│
├── :z : true
│
└── :q
    │
    ├── :a : true
    │
    └── :b : false


-2.0794415416798357

538

In [15]:
debugIO = open("read.txt", "w+")
logger = ConsoleLogger(debugIO, Debug)
global_logger(logger)

io = read_from_file("data.trace")
recovered_tr = try
    recovered_tr = GenArrow._deserialize_lazy(io)
    close(debugIO)
    recovered_tr
catch
    close(debugIO)
    rethrow()
end
# display(get_score(recovered_tr))
# display(get_choices(recovered_tr))
# display(get_args(recovered_tr))

LazyTrace(Trie{Any, Gen.ChoiceOrCallRecord}(Dict{Any, Gen.ChoiceOrCallRecord}(:z => Gen.ChoiceOrCallRecord{Bool}(true, -0.6931471805599453, NaN, true), :q => Gen.ChoiceOrCallRecord{LazyTrace}(LazyTrace(Trie{Any, Gen.ChoiceOrCallRecord}(Dict{Any, Gen.ChoiceOrCallRecord}(:a => Gen.ChoiceOrCallRecord{Bool}(true, -0.6931471805599453, NaN, true), :b => Gen.ChoiceOrCallRecord{Bool}(false, -0.6931471805599453, NaN, true)), Dict{Any, Trie{Any, Gen.ChoiceOrCallRecord}}()), false, -2.772588722239781, 0.0, ("what the",), false), -2.772588722239781, 0.0, false)), Dict{Any, Trie{Any, Gen.ChoiceOrCallRecord}}()), false, -5.545177444479562, 0.0, (10,), false)

### Internal Nodes

In [16]:
@gen function model()
    {:x=>1} ~ bernoulli(0.5)
    {:x=>2} ~ categorical([0.25, 0.25, 0.25, 0.25])
    return 1
end

DynamicDSLFunction{Any}(Dict{Symbol, Any}(), Dict{Symbol, Any}(), Type[], false, Union{Nothing, Some{Any}}[], var"##model#317", Bool[], false)

In [17]:
debugIO = open("write.txt", "w+")
logger = ConsoleLogger(debugIO, Debug)
global_logger(logger)
tr_old, w_old = generate(model, ())
display(get_choices(tr_old))
display(get_score(tr_old))
io = IOBuffer()
GenArrow.serialize(io, tr_old)
close(debugIO)
write_to_file("data.trace", io)

│
└── :x
    │
    ├── 2 : 1
    │
    └── 1 : false


-2.0794415416798357

332

In [18]:
debugIO = open("read.txt", "w+")
logger = ConsoleLogger(debugIO, Debug)
global_logger(logger)
seekstart(io)
recovered_tr = try
    recovered_tr = GenArrow._deserialize_lazy(io)
    close(debugIO)
    recovered_tr
catch
    close(debugIO)
    rethrow()
end
display(get_choices(recovered_tr))
display(get_score(recovered_tr))

│
└── :x
    │
    ├── 2 : 1
    │
    └── 1 : false


-4.1588830833596715

### Conversion

In [19]:
@gen function mixed_submodel(w)
    a ~ bernoulli(0.5)
    b ~ bernoulli(0.5)
end

@gen function mixed(n)
    z ~ bernoulli(0.5)
    {:x=>1} ~ bernoulli(0.5)
    {:x=>2} ~ categorical([0.25, 0.25, 0.25, 0.25])
    q ~ mixed_submodel("hol up")
end

DynamicDSLFunction{Any}(Dict{Symbol, Any}(), Dict{Symbol, Any}(), Type[Any], false, Union{Nothing, Some{Any}}[nothing], var"##mixed#319", Bool[0], false)

In [20]:
debugIO = open("write.txt", "w+")
logger = ConsoleLogger(debugIO, Debug)
global_logger(logger)
tr_old, w_old = generate(mixed, (10,))
display(get_choices(tr_old))
display(get_score(tr_old))
io = IOBuffer()
GenArrow.serialize(io, tr_old)
close(debugIO)
write_to_file("data.trace", io)

MethodError: MethodError: no method matching var"##model#317"(::Gen.GFGenerateState, ::Int64)
Closest candidates are:
  var"##model#317"(::Any) at ~/Documents/probcomp/GenArrow.jl/test/unit/lazy.ipynb:1

In [21]:
debugIO = open("read.txt", "w+")
logger = ConsoleLogger(debugIO, Debug)
global_logger(logger)

io = read_from_file("data.trace")
recovered_tr = try
    recovered_tr = GenArrow._deserialize_lazy(io, gen_fn = mixed)
    close(debugIO)
    recovered_tr
catch
    close(debugIO)
    rethrow()
end
# display(get_score(recovered_tr))
# display(get_choices(recovered_tr))
# display(get_args(recovered_tr))

LazyTrace(Trie{Any, Gen.ChoiceOrCallRecord}(Dict{Any, Gen.ChoiceOrCallRecord}(), Dict{Any, Trie{Any, Gen.ChoiceOrCallRecord}}(:x => Trie{Any, Gen.ChoiceOrCallRecord}(Dict{Any, Gen.ChoiceOrCallRecord}(2 => Gen.ChoiceOrCallRecord{Int64}(1, -1.3862943611198906, NaN, true), 1 => Gen.ChoiceOrCallRecord{Bool}(false, -0.6931471805599453, NaN, true)), Dict{Any, Trie{Any, Gen.ChoiceOrCallRecord}}()))), false, -4.1588830833596715, 0.0, (), 1)